# List of Weather Station for Wunderground

**Kuala Pilah & Alor Gajah** 
Location: MALACCA AIRPORT STATION https://www.wunderground.com/history/daily/my/malacca/WMKM/date/


**Jengka & Gambang** 
Location: KUANTAN AIRPORT STATION https://www.wunderground.com/history/daily/my/kuantan/WMKD/date/ 


**Permatang Pauh**
Location: PENANG INTERNATIONAL AIRPORT STATION https://www.wunderground.com/history/daily/my/bayan-lepas/WMKP/date/

**Segamat**
Location: SEGAMAT AIRPORT STATION https://www.wunderground.com/history/daily/my/segamat/WMAZ/date/

**Dungun**
  Location: KERTEH AIRPORT STATION https://www.wunderground.com/history/daily/my/kerteh/WMKE/date/

# Coding 

In [1]:
from bs4 import BeautifulSoup as BS
from datetime import datetime, timedelta
from selenium import webdriver
from functools import reduce
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
                

In [2]:

def render_page(url,type):
        options=webdriver.ChromeOptions()
        options.add_argument('--headless')
        driver = webdriver.Chrome(options=options)
        driver.get(url)
        time.sleep(5)
        if type =="C":
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, 'wuSettings'))
            )
            element.click()
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="wuSettings-quick"]/div/a[2]')))
            element.click()
            time.sleep(3)
            r = driver.page_source
            driver.quit()
        if type=="F":
            r = driver.page_source
            driver.quit()
        return r

In [3]:
def hourly_scraper(page, dates, type):
    all_data = []  # List to store each day's DataFrame

    for d in dates:
        url = f"{page}{d}"
        r = render_page(url, type)
        soup = BS(r, "html.parser")
        container = soup.find('lib-city-history-observation')
        check = container.find('tbody')

        data = []
        data_hour = []
        for i in check.find_all('span', class_='ng-star-inserted'):
            trial = i.get_text()
            data_hour.append(trial)

        for i in check.find_all('span', class_='wu-value wu-value-to'):
            trial = i.get_text()
            data.append(trial)

        numbers = pd.DataFrame([data[i:i+7] for i in range(0, len(data), 7)], columns=["Temperature", "Dew Point", "Humidity", "Wind Speed", "Wind Gust", "Pressure", "Precipitation"])
        hour = pd.DataFrame(data_hour[0::17], columns=["Time"])
        wind = pd.DataFrame(data_hour[7::17], columns=["Wind"])
        condition = pd.DataFrame(data_hour[16::17], columns=["Condition"])

        dfs = [hour, numbers, wind, condition]

        df_final = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True), dfs)
        df_final['Date'] = d

        all_data.append(df_final)  # Append each day's DataFrame to the list

        print(f"{d} finished!")

    output = pd.concat(all_data, ignore_index=True)  # Concatenate all DataFrames into a single DataFrame
    return output


#####  Please change the base_url according to the Wunderground location 

In [9]:
## -- 2/7/2024 4:01 pm
# Define the base URL and start date
base_url = "https://www.wunderground.com/history/daily/my/malacca/WMKM/date/"
start_date = datetime(2024, 7, 1)

# Define the number of days to iterate through
num_days = 31
all_data = pd.DataFrame()  # Initialize an empty DataFrame to concatenate results

# Loop through the dates and call the hourly_scraper function
for i in range(num_days):
    current_date = start_date + timedelta(days=i)
    formatted_date = current_date.strftime("%Y-%m-%d")
    
    # Call the hourly_scraper function for the current date
    hourly1 = hourly_scraper(base_url, [formatted_date], "F")

    # Concatenate the result to the all_data DataFrame
    all_data = pd.concat([all_data, hourly1], ignore_index=True)

    # Optionally, add a delay to avoid overloading the website
    import time
    time.sleep(10)  # Sleep for 5 seconds between requests (adjust as needed)

# Save all data to a single CSV file if needed
all_data.to_csv('July_2024_KP.csv', index=False)


import winsound

# Your long-running code here

# Play a sound when done
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)

2024-07-01 finished!
2024-07-02 finished!
2024-07-03 finished!
2024-07-04 finished!
2024-07-05 finished!
2024-07-06 finished!
2024-07-07 finished!
2024-07-08 finished!
2024-07-09 finished!
2024-07-10 finished!
2024-07-11 finished!
2024-07-12 finished!
2024-07-13 finished!
2024-07-14 finished!
2024-07-15 finished!
2024-07-16 finished!
2024-07-17 finished!
2024-07-18 finished!
2024-07-19 finished!
2024-07-20 finished!
2024-07-21 finished!
2024-07-22 finished!
2024-07-23 finished!
2024-07-24 finished!
2024-07-25 finished!
2024-07-26 finished!
2024-07-27 finished!
2024-07-28 finished!
2024-07-29 finished!
2024-07-30 finished!
2024-07-31 finished!


In [8]:
all_data.to_csv('Jan_17July_2024_KP.csv', index=False)